# 시장데이터 종가 생성

1. key 생성 : 거래소코드+년도
2. 종가변동률 구하기


In [116]:
import pandas as pd
import numpy as np
from tqdm import tqdm 

import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
plt.rcParams['font.family'] = 'malgun gothic'

from sklearn.metrics import accuracy_score, log_loss, plot_confusion_matrix, recall_score, precision_score, f1_score, roc_auc_score,average_precision_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from catboost import CatBoostClassifier, Pool  
from lightgbm import LGBMClassifier   

from imblearn.under_sampling import ClusterCentroids, RandomUnderSampler

import warnings
warnings.filterwarnings(action='ignore')

## 데이터 불러오기

In [117]:
price = pd.read_csv(r'..\..\..\Code\SH\csv\0020종가.csv', encoding= 'euc-kr')

In [118]:
price

,회사명,거래소코드,회계년도,외국인_주식분포비율,종가
0,(주)CMG제약,58820,2000/12,NaN,NaN
1,(주)CMG제약,58820,2001/12,0.00,1420.0
2,(주)CMG제약,58820,2002/12,0.00,520.0
3,(주)CMG제약,58820,2003/12,0.00,800.0
4,(주)CMG제약,58820,2004/12,0.00,550.0
...,...,...,...,...,...
27160,흥구석유(주),24060,2016/12,0.97,3425.0
27161,흥구석유(주),24060,2017/12,0.03,2775.0
27162,흥구석유(주),24060,2018/12,1.73,2705.0
27163,흥구석유(주),24060,2019/12,0.56,5360.0


In [119]:
price.isna().sum()

회사명              0
거래소코드            0
회계년도             0
외국인_주식분포비율    5369
종가            6379
dtype: int64

In [120]:
#일단 결측치 fillna로 0채워줌
price = price.fillna(0)

## key 생성
거래소코드+년도

In [121]:
price['회계년도'].str[:4]

0        2000
1        2001
2        2002
3        2003
4        2004
         ... 
27160    2016
27161    2017
27162    2018
27163    2019
27164    2020
Name: 회계년도, Length: 27165, dtype: object

In [122]:
price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27165 entries, 0 to 27164
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   회사명         27165 non-null  object 
 1   거래소코드       27165 non-null  int64  
 2   회계년도        27165 non-null  object 
 3   외국인_주식분포비율  27165 non-null  float64
 4   종가          27165 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.0+ MB


In [123]:
price['년'] = price['회계년도'].str[:4].astype(int)

In [124]:
price['key'] = price['거래소코드'].astype(str) + (price['년']+1).astype(str)

In [125]:
price

,회사명,거래소코드,회계년도,외국인_주식분포비율,종가,년,key
0,(주)CMG제약,58820,2000/12,0.00,0.0,2000,588202001
1,(주)CMG제약,58820,2001/12,0.00,1420.0,2001,588202002
2,(주)CMG제약,58820,2002/12,0.00,520.0,2002,588202003
3,(주)CMG제약,58820,2003/12,0.00,800.0,2003,588202004
4,(주)CMG제약,58820,2004/12,0.00,550.0,2004,588202005
...,...,...,...,...,...,...,...
27160,흥구석유(주),24060,2016/12,0.97,3425.0,2016,240602017
27161,흥구석유(주),24060,2017/12,0.03,2775.0,2017,240602018
27162,흥구석유(주),24060,2018/12,1.73,2705.0,2018,240602019
27163,흥구석유(주),24060,2019/12,0.56,5360.0,2019,240602020


## 종가변동률 구하기

In [126]:
#종가변동률 구하는 식 <br>
#(df['자기자본배율'] - df['자기자본배율'].shift(1))/df['자기자본배율'].shift(1)*100

In [127]:
# for문으로 생성해주기
#종가변동률 구하는 식 <br>
#(df['자기자본배율'] - df['자기자본배율'].shift(1))/df['자기자본배율'].shift(1)*100
company = price['회사명'].unique().tolist()

for i in tqdm(range(len(company))):
    price.loc[price['회사명']==company[i], '종가변동률']=(price.loc[price['회사명']==company[i]]['종가'] - price.loc[price['회사명']==company[i]]['종가'].shift(1))/price.loc[price['회사명']==company[i]]['종가'].shift(1)
    price = price.fillna(0)
    price['종가변동률'] = price['종가변동률'].replace([np.inf], np.nan)
    price['종가변동률'] = price['종가변동률'].fillna(method='bfill')# 후년 데이터로 대체
    

100%|██████████| 2020/2020 [00:23<00:00, 85.66it/s]


In [186]:
price[price['종가변동률'] == [np.inf]]

ValueError: ('Lengths must match to compare', (27165,), (1,))

In [128]:
price

,회사명,거래소코드,회계년도,외국인_주식분포비율,종가,년,key,종가변동률
0,(주)CMG제약,58820,2000/12,0.00,0.0,2000,588202001,0.000000
1,(주)CMG제약,58820,2001/12,0.00,1420.0,2001,588202002,-0.633803
2,(주)CMG제약,58820,2002/12,0.00,520.0,2002,588202003,-0.633803
3,(주)CMG제약,58820,2003/12,0.00,800.0,2003,588202004,0.538462
4,(주)CMG제약,58820,2004/12,0.00,550.0,2004,588202005,-0.312500
...,...,...,...,...,...,...,...,...
27160,흥구석유(주),24060,2016/12,0.97,3425.0,2016,240602017,0.126645
27161,흥구석유(주),24060,2017/12,0.03,2775.0,2017,240602018,-0.189781
27162,흥구석유(주),24060,2018/12,1.73,2705.0,2018,240602019,-0.025225
27163,흥구석유(주),24060,2019/12,0.56,5360.0,2019,240602020,0.981516


In [129]:
price.loc[price['회사명']==company[10]]

,회사명,거래소코드,회계년도,외국인_주식분포비율,종가,년,key,종가변동률
129,(주)고영테크놀러지,98460,2005/12,0.00,0.0,2005,984602006,0.000000
130,(주)고영테크놀러지,98460,2006/12,0.00,0.0,2006,984602007,0.000000
131,(주)고영테크놀러지,98460,2007/12,0.00,0.0,2007,984602008,0.000000
132,(주)고영테크놀러지,98460,2008/12,4.24,6990.0,2008,984602009,0.530758
133,(주)고영테크놀러지,98460,2009/12,0.75,10700.0,2009,984602010,0.530758
134,(주)고영테크놀러지,98460,2010/12,5.41,18000.0,2010,984602011,0.682243
135,(주)고영테크놀러지,98460,2011/12,23.39,30900.0,2011,984602012,0.716667
136,(주)고영테크놀러지,98460,2012/12,17.01,28100.0,2012,984602013,-0.090615
137,(주)고영테크놀러지,98460,2013/12,17.83,27950.0,2013,984602014,-0.005338
138,(주)고영테크놀러지,98460,2014/12,38.70,42800.0,2014,984602015,0.531306


In [130]:
# 결측치 확인
price.isna().sum()

회사명           0
거래소코드         0
회계년도          0
외국인_주식분포비율    0
종가            0
년             0
key           0
종가변동률         0
dtype: int64

In [131]:
#csv저장
#price.to_csv('0020종가변동률 반영.csv')

## finaldataset과 merge

In [132]:
df_ = pd.read_csv(r'..\..\..\Data\최종데이터셋\Finaldataset_ver7.1.csv', index_col=0)
df = df_
df

,회사명,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자기자본배당률,금융비용 대 부채비율,자기자본배율,유동부채비율,...,공시발생일,정기공시제목,수시공시제목,대표이사 변경,최대주주 변경,회계처리위반,횡령배임,폐지사유요약합본,신종채권,영업조업중단
0,(주)CMG제약,58820.0,2000/12,5.0,0,UQ,0.00,0.019074,179.37,131.99,...,0,0,0,0.0,0.0,0.0,0.0,['N'],0.0,0.0
1,(주)CMG제약,58820.0,2001/12,5.0,0,UQ,0.00,0.023517,179.37,131.99,...,0,0,0,0.0,0.0,0.0,0.0,['N'],0.0,0.0
2,(주)CMG제약,58820.0,2002/12,5.0,0,UQ,0.00,0.043081,133.37,132.37,...,0,0,0,1.0,0.0,0.0,0.0,['N'],0.0,1.0
3,(주)CMG제약,58820.0,2003/12,5.0,0,UQ,0.00,0.058531,131.30,71.46,...,0,0,0,1.0,0.0,0.0,0.0,['N'],0.0,0.0
4,(주)CMG제약,58820.0,2004/12,5.0,0,UQ,0.00,0.047346,52.25,101.56,...,0,0,0,1.0,0.0,0.0,0.0,['N'],0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25119,흥구석유(주),24060.0,2015/12,5.0,0,UQ,1.70,0.005597,4548.48,4.01,...,0,0,0,0.0,0.0,0.0,0.0,['N'],0.0,0.0
25120,흥구석유(주),24060.0,2016/12,5.0,0,UQ,2.10,0.012651,4611.84,5.19,...,0,0,0,0.0,0.0,0.0,0.0,['N'],0.0,0.0
25121,흥구석유(주),24060.0,2017/12,5.0,0,UQ,2.08,0.012502,4650.29,12.83,...,0,0,0,0.0,0.0,0.0,0.0,['N'],0.0,0.0
25122,흥구석유(주),24060.0,2018/12,5.0,0,UQ,2.48,0.018056,4696.15,5.40,...,0,0,0,0.0,0.0,0.0,0.0,['N'],0.0,0.0


In [133]:
df_.shape

(25124, 30)

In [134]:
df['부도'].value_counts()

1.0    24722
0.0      402
Name: 부도, dtype: int64

In [135]:
#부도를 1 정상기업 0으로 처리
df['부도'] = df['부도'].replace({ 1 : 0, 0 : 1})
df['부도'].value_counts()

0.0    24722
1.0      402
Name: 부도, dtype: int64

In [136]:
df['key'] = df['key'].astype(float).astype(int).astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25124 entries, 0 to 25123
Data columns (total 30 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   회사명          25124 non-null  object 
 1   거래소코드        25124 non-null  float64
 2   회계년도         25124 non-null  object 
 3   소속코드         25124 non-null  float64
 4   상장폐지일        25124 non-null  object 
 5   감사의견코드       25124 non-null  object 
 6   자기자본배당률      25124 non-null  float64
 7   금융비용 대 부채비율  25124 non-null  float64
 8   자기자본배율       25124 non-null  float64
 9   유동부채비율       25124 non-null  float64
 10  매입채무회전률      25124 non-null  float64
 11  유동자산회전률      25124 non-null  float64
 12  총자본투자효율      25124 non-null  float64
 13  순운전자본비율      25124 non-null  float64
 14  누적수익성비율      25124 non-null  float64
 15  총자산영업이익률     25124 non-null  float64
 16  총자산회전율       25124 non-null  float64
 17  년            25124 non-null  float64
 18  key          25124 non-null  object 
 19  부도  

In [137]:
price.columns
price_col = [ '외국인_주식분포비율', '종가', 'key', '종가변동률']

In [138]:
df

,회사명,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자기자본배당률,금융비용 대 부채비율,자기자본배율,유동부채비율,...,공시발생일,정기공시제목,수시공시제목,대표이사 변경,최대주주 변경,회계처리위반,횡령배임,폐지사유요약합본,신종채권,영업조업중단
0,(주)CMG제약,58820.0,2000/12,5.0,0,UQ,0.00,0.019074,179.37,131.99,...,0,0,0,0.0,0.0,0.0,0.0,['N'],0.0,0.0
1,(주)CMG제약,58820.0,2001/12,5.0,0,UQ,0.00,0.023517,179.37,131.99,...,0,0,0,0.0,0.0,0.0,0.0,['N'],0.0,0.0
2,(주)CMG제약,58820.0,2002/12,5.0,0,UQ,0.00,0.043081,133.37,132.37,...,0,0,0,1.0,0.0,0.0,0.0,['N'],0.0,1.0
3,(주)CMG제약,58820.0,2003/12,5.0,0,UQ,0.00,0.058531,131.30,71.46,...,0,0,0,1.0,0.0,0.0,0.0,['N'],0.0,0.0
4,(주)CMG제약,58820.0,2004/12,5.0,0,UQ,0.00,0.047346,52.25,101.56,...,0,0,0,1.0,0.0,0.0,0.0,['N'],0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25119,흥구석유(주),24060.0,2015/12,5.0,0,UQ,1.70,0.005597,4548.48,4.01,...,0,0,0,0.0,0.0,0.0,0.0,['N'],0.0,0.0
25120,흥구석유(주),24060.0,2016/12,5.0,0,UQ,2.10,0.012651,4611.84,5.19,...,0,0,0,0.0,0.0,0.0,0.0,['N'],0.0,0.0
25121,흥구석유(주),24060.0,2017/12,5.0,0,UQ,2.08,0.012502,4650.29,12.83,...,0,0,0,0.0,0.0,0.0,0.0,['N'],0.0,0.0
25122,흥구석유(주),24060.0,2018/12,5.0,0,UQ,2.48,0.018056,4696.15,5.40,...,0,0,0,0.0,0.0,0.0,0.0,['N'],0.0,0.0


In [139]:
#merge 진행
df = df.merge(price[price_col], how = 'outer', on = 'key')

#해당되지 않는 데이터 삭제를 위해 fillna 처리
df['횡령배임'] = df['횡령배임'].fillna(0)

#해당되지 않는 데이터 삭제 
df.dropna(inplace=True)
df.drop(columns=['폐지사유요약합본'], inplace=True)
df

,회사명,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자기자본배당률,금융비용 대 부채비율,자기자본배율,유동부채비율,...,수시공시제목,대표이사 변경,최대주주 변경,회계처리위반,횡령배임,신종채권,영업조업중단,외국인_주식분포비율,종가,종가변동률
0,(주)CMG제약,58820.0,2000/12,5.0,0,UQ,0.00,0.019074,179.37,131.99,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000
1,(주)CMG제약,58820.0,2001/12,5.0,0,UQ,0.00,0.023517,179.37,131.99,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1420.0,-0.633803
2,(주)CMG제약,58820.0,2002/12,5.0,0,UQ,0.00,0.043081,133.37,132.37,...,0,1.0,0.0,0.0,0.0,0.0,1.0,0.00,520.0,-0.633803
3,(주)CMG제약,58820.0,2003/12,5.0,0,UQ,0.00,0.058531,131.30,71.46,...,0,1.0,0.0,0.0,0.0,0.0,0.0,0.00,800.0,0.538462
4,(주)CMG제약,58820.0,2004/12,5.0,0,UQ,0.00,0.047346,52.25,101.56,...,0,1.0,0.0,0.0,0.0,0.0,0.0,0.00,550.0,-0.312500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25119,흥구석유(주),24060.0,2015/12,5.0,0,UQ,1.70,0.005597,4548.48,4.01,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.60,3040.0,0.404157
25120,흥구석유(주),24060.0,2016/12,5.0,0,UQ,2.10,0.012651,4611.84,5.19,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.97,3425.0,0.126645
25121,흥구석유(주),24060.0,2017/12,5.0,0,UQ,2.08,0.012502,4650.29,12.83,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.03,2775.0,-0.189781
25122,흥구석유(주),24060.0,2018/12,5.0,0,UQ,2.48,0.018056,4696.15,5.40,...,0,0.0,0.0,0.0,0.0,0.0,0.0,1.73,2705.0,-0.025225


In [140]:
df.isna().sum()

회사명            0
거래소코드          0
회계년도           0
소속코드           0
상장폐지일          0
감사의견코드         0
자기자본배당률        0
금융비용 대 부채비율    0
자기자본배율         0
유동부채비율         0
매입채무회전률        0
유동자산회전률        0
총자본투자효율        0
순운전자본비율        0
누적수익성비율        0
총자산영업이익률       0
총자산회전율         0
년              0
key            0
부도             0
공시발생일          0
정기공시제목         0
수시공시제목         0
대표이사 변경        0
최대주주 변경        0
회계처리위반         0
횡령배임           0
신종채권           0
영업조업중단         0
외국인_주식분포비율     0
종가             0
종가변동률          0
dtype: int64

## 데이터 저장

In [141]:
df.to_csv('finaldataset_ver7.2.csv')